In [214]:
from gurobipy import *
import pandas as pd
import math
import numpy as np

In [215]:
def read_data():
    drive_data = pd.read_csv('Adams_100_1_Fastest.csv')

    drive_distance = {}
    drive_time = {}
    arc = drive_data['key'].values
    distance = drive_data['Distance'].values
    time = drive_data['Time'].values

    n = len(arc)
    for i in range(0,n):
        a = re.sub("\(" , "", arc[i])
        a = re.sub("\)", "", a)
        a,b = a.split(', ')
        drive_distance[(int(a),int(b))]= distance[i]
        drive_time[(int(a),int(b))] = time[i]

    walk_data = pd.read_csv('Adams_100_1_Walking.csv')

    walk_distance = {}
    walk_time = {}
    arc = walk_data['key'].values
    distance = walk_data['Distance'].values
    time = walk_data['Time'].values
    n = len(arc)
    for i in range(0,n):
        a = re.sub("\(" , "", arc[i])
        a = re.sub("\)", "", a)
        a,b = a.split(', ')
        walk_distance[(int(a),int(b))]= distance[i]
        walk_time[(int(a),int(b))] = time[i]

    return int(math.sqrt(n)), drive_distance, drive_time, walk_distance, walk_time




In [216]:
f = 2.8 # time for loading packages
q = 3 # capacity

# generate set
def serve_set(n):
    serve_set = set(())
    for i in range(1, n + 1):
        serve_set.add((i,))
        for j in range(i + 1, n + 1):
            serve_set.add((i, j)) # tuple
            for k in range(j + 1, n + 1):
                serve_set.add((i, j, k))
    m = len(serve_set)  # m: number of serve sets
    return m, serve_set




In [217]:
# Testing whether to add the equalities

# for Claim 5,6
# D(i,k) <= W(i,k) for all i,k \in C
def D_less_equal_than_W(drive_time, walk_time, n):
    temp = True
    for i in range(1, n+1):
        for k in range(1, n+1):
            if drive_time[(i, k)]>walk_time[(i,k)]:
                temp = False
                break
        if not temp:
            break
    return temp

# for Corollary 1
# Consider i ∈ C. If D(i, k) + f <= W(i, k) and D(k,i)+f <= W(k,i) for all k\in C\{i}
def D_plus_f_less_equal_than_W(i, drive_time, walk_time, f, n):
    temp = True
    for k in range(1, n+1):
        if k != i:
            if drive_time[(i,k)] + f > walk_time[(i,k)]:
                temp = False
                break
            elif drive_time[(k,i)] + f > walk_time[(k,i)]:
                temp = False
                break
    return temp

# for Claim 2 
# σ ∈ S such that D(i, k) ≤ W(i, k) and D(k,i)≤W(k,i) for all i∈σ and k∈C
def D_less_equal_than_W_for_i_in_sigma(sigma, drive_time, walk_time, n):
    temp = True
    for i in sigma:
        for k in range(1, n+1):
            if drive_time[(i, k)] > walk_time[(i,k)]:
                temp = False
            elif drive_time[(k, i)] > walk_time[(k,i)]:
                temp = False
    return temp

In [218]:
def find_shortest_time(i, sigma, k):
    if len(sigma) == 1:
        w1 = walk_time[(i, sigma[0])] + walk_time[(sigma[0], k)]
        min_w = w1
        
    elif len(sigma) == 2:
        inx_order_list = [(0,1), (1,0)]
    
        w1 = walk_time[(i, sigma[0])] + walk_time[(sigma[0], sigma[1])] + walk_time[(sigma[1], k)]
        w2 = walk_time[(i, sigma[1])] + walk_time[(sigma[1], sigma[0])] + walk_time[(sigma[0], k)]
        
        min_w = np.min([w1,w2])
        
    elif len(sigma) == 3:
        inx_order_list = [(0,1,2),(0,2,1),(1,0,2),(1,2,0),(2,0,1),(2,1,0)]

        w1 = walk_time[(i, sigma[0])] + walk_time[(sigma[0], sigma[1])] + walk_time[(sigma[1], sigma[2])] + walk_time[(sigma[2], k)]
        w2 = walk_time[(i, sigma[0])] + walk_time[(sigma[0], sigma[2])] + walk_time[(sigma[2], sigma[1])] + walk_time[(sigma[1], k)]
        w3 = walk_time[(i, sigma[1])] + walk_time[(sigma[1], sigma[0])] + walk_time[(sigma[0], sigma[2])] + walk_time[(sigma[2], k)]
        w4 = walk_time[(i, sigma[1])] + walk_time[(sigma[1], sigma[2])] + walk_time[(sigma[2], sigma[0])] + walk_time[(sigma[0], k)]
        w5 = walk_time[(i, sigma[2])] + walk_time[(sigma[2], sigma[0])] + walk_time[(sigma[0], sigma[1])] + walk_time[(sigma[1], k)]
        w6 = walk_time[(i, sigma[2])] + walk_time[(sigma[2], sigma[1])] + walk_time[(sigma[1], sigma[0])] + walk_time[(sigma[0], k)]

        min_w = np.min([w1, w2, w3, w4, w5, w6])
        
    return min_w

def find_shortest_path(i, sigma, k):
    if len(sigma) == 1:
        c_first = sigma[0]
        c_last = sigma[0]
    elif len(sigma) == 2:
        inx_order_list = [(0,1), (1,0)]
        
        w1 = walk_time[(i, sigma[0])] + walk_time[(sigma[0], sigma[1])] + walk_time[(sigma[1], k)]
        w2 = walk_time[(i, sigma[1])] + walk_time[(sigma[1], sigma[0])] + walk_time[(sigma[0], k)]
        
        temp_tuple = inx_order_list[np.argmin([w1,w2])]
        c_first = sigma[temp_tuple[0]]
        c_last = sigma[temp_tuple[1]]
        
    elif len(sigma) == 3:
        inx_order_list = [(0,1,2),(0,2,1),(1,0,2),(1,2,0),(2,0,1),(2,1,0)]
        
        w1 = walk_time[(i, sigma[0])] + walk_time[(sigma[0], sigma[1])] + walk_time[(sigma[1], sigma[2])] + walk_time[(sigma[2], k)]
        w2 = walk_time[(i, sigma[0])] + walk_time[(sigma[0], sigma[2])] + walk_time[(sigma[2], sigma[1])] + walk_time[(sigma[1], k)]
        w3 = walk_time[(i, sigma[1])] + walk_time[(sigma[1], sigma[0])] + walk_time[(sigma[0], sigma[2])] + walk_time[(sigma[2], k)]
        w4 = walk_time[(i, sigma[1])] + walk_time[(sigma[1], sigma[2])] + walk_time[(sigma[2], sigma[0])] + walk_time[(sigma[0], k)]
        w5 = walk_time[(i, sigma[2])] + walk_time[(sigma[2], sigma[0])] + walk_time[(sigma[0], sigma[1])] + walk_time[(sigma[1], k)]
        w6 = walk_time[(i, sigma[2])] + walk_time[(sigma[2], sigma[1])] + walk_time[(sigma[1], sigma[0])] + walk_time[(sigma[0], k)]

        temp_tuple = inx_order_list[np.argmin([w1, w2, w3, w4, w5, w6])]
        c_first = sigma[temp_tuple[0]]
        c_last = sigma[temp_tuple[2]]
        
    return c_first, c_last

# compute w_i_sigma_k
def w_with_set(serve_list, walk_time, n):
    w = {}
    for i in range(1, n + 1):
        for k in range(1, n + 1):
            for sigma in serve_list:
                index = serve_list.index(sigma)
                w[(i,index,k)] = find_shortest_time(i, sigma, k)            
    return w



In [219]:
def compare_d_w(w, drive_time):
    wait_time = {}
    for w_route , w_time in w.items():
        d_time = drive_time[(w_route[0],w_route[2])]
        wait_time[w_route] = max(d_time - w_time,0)
    return wait_time


In [220]:
# n: number of costumers, not include depot
n, drive_distance, drive_time, walk_distance, walk_time = read_data()

In [200]:
n = 20

In [201]:
# serve set
m, serve_set = serve_set(n)  # m: |S|, length of serve_set
serve_list = list(serve_set)

In [202]:
#  w_i_sigma_k，这一步挺慢的
w_with_set = w_with_set(serve_list, walk_time, n)

In [203]:
# max term in obj function
wait_time = compare_d_w(w_with_set, drive_time)

In [204]:
model = Model("mip")
# setParam, not sure 先这么写了
model.setParam('Timelimit', 3600)



Changed value of parameter Timelimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [205]:

# add variables
x = {}
for i in range(n+1):
    for k in range(n+1):
        name = 'x_' + str(i) + '_' + str(k)
        x[i,k] = model.addVar(vtype=GRB.BINARY, name=name)

y = {}
for i in range(1,n+1):
    for s_id in range(m):
        for k in range(1,n+1):
            name = 'y_' + str(i) + '_' + str(serve_list[s_id]) + '_' + str(k)
            y[i,s_id,k] = model.addVar(vtype=GRB.BINARY, name=name)

v = {}
for i in range(n+1):
    for k in range(1,n+1):
        if i != k:
            name = 'v_' + str(i) + '_' + str(k)
            v[i,k] = model.addVar(lb = 0, vtype=GRB.INTEGER, name=name) # not sure include n




In [206]:
# objective function
obj = LinExpr(0)

for i in range(n+1):
    for k in range(n+1):
        if i != k:
            obj.add(x[i,k]*drive_time[(i,k)])

for i in range(1,n+1):
    for k in range(1,n+1):
        for sigma in serve_list:
            index = serve_list.index(sigma)
            obj.add(y[i,index,k] * (f + w_with_set[(i, index, k)] + wait_time[(i, index, k)]))

model.setObjective(obj, GRB.MINIMIZE)


In [207]:
# Constraints

for i in range(n+1):
    model.addConstr(x[i,i] == 0,  name= 'Constraint 1_'+str(i))


# Constraint 2
obj_c_2 = LinExpr(0)
for i in range(1,n+1):
    obj_c_2.add(x[i,0])
model.addConstr(obj_c_2 == 1,  name= 'Constraint 2')

# Constraint 3
obj_c_3 = LinExpr(0)
for i in range(1,n+1):
    obj_c_3.add(x[0,i])
model.addConstr(obj_c_3 == 1,  name= 'Constraint 3')


<gurobi.Constr *Awaiting Model Update*>

In [208]:
# Constraint 4
for l in range(1,n+1):
    obj_c_4 = LinExpr(0)
    for i in range(1, n + 1):
        for k in range(1, n + 1):
            for sigma in serve_list:
                if l in sigma:
                    index = serve_list.index(sigma)
                    obj_c_4.add(y[i,index,k])
    model.addConstr(obj_c_4 == 1,  name= 'Constraint 4_'+str(l))

# Constraint 5
for i in range(1,n+1):
    obj_c_5_1 = LinExpr(0)
    for k in range(1, n + 1):
        for s_id in range(len(serve_list)):
            obj_c_5_1.add(y[i,s_id,k])
    obj_c_5_2 = LinExpr(0)
    for l in range(n + 1):
        obj_c_5_2.add(x[l,i])
    model.addConstr(obj_c_5_1 == obj_c_5_2,  name= 'Constraint 5_'+str(i))

# Constraint 6
for k in range(1,n+1):
    obj_c_6_1 = LinExpr(0)
    for i in range(1, n + 1):
        for s_id in range(len(serve_list)):
            obj_c_6_1.add(y[i,s_id,k])
    obj_c_6_2 = LinExpr(0)
    for l in range(n + 1):
        obj_c_6_2.add(x[k,l])
    model.addConstr(obj_c_6_1 == obj_c_6_2,  name= 'Constraint 6_'+str(k))

In [209]:
# Constraint 7
obj_c_7 = LinExpr(0)
for k in range(1,n+1):
    obj_c_7.add(v[0,k])
model.addConstr(obj_c_7 == n,  name= 'Constraint 7')

<gurobi.Constr *Awaiting Model Update*>

In [210]:
# Constraint 8
for i in range(1,n+1):
    model.addConstr(v[0,i] <= n * x[0,i],  name= 'Constraint 8_'+str(i))

In [211]:
# Constraint 9
for i in range(1,n+1):
    for k in range(1, n + 1):
        if i != k:
            obj_c_9 = LinExpr(0)
            for s_id in range(len(serve_list)):
                obj_c_9.add(y[i,s_id,k])
            model.addConstr((obj_c_9 + x[i,k]) * n >= v[i,k], name='Constraint 9_'+str(i)+'_'+str(k))
            

In [212]:

# Constraint 10
for i in range(1,n+1):
    obj_c_10 = LinExpr(0)
    # first term
    for k in range(n+1):
        if i != k:
            obj_c_10.add(v[k, i])
    # second term
    for k in range(1,n+1):
        if i != k:
            obj_c_10.add(-v[i, k])
    # third term
    for k in range(1,n+1):
        obj_c_10.add(-x[i, k])
    # forth term
    for k in range(1,n+1):
        for sigma in serve_list:
            index = serve_list.index(sigma)
            obj_c_10.add(-(len(sigma)-1)*y[i,index,k])
    model.addConstr(obj_c_10 == 0, name='Constraint 10_'+str(i))


In [ ]:
'''
# Service Set Reduction

# Corollary 1
for i in range(1,n+1):
    if D_plus_f_less_equal_than_W(i, drive_time, walk_time, f, n):
        obj_claim_1_ki = LinExpr(0)
        obj_claim_1_ik = LinExpr(0)
        for k in range(n+1):
            obj_claim_1_ki.add(x[k,i])
            obj_claim_1_ik.add(x[i,k])     
        model.addConstr( obj_claim_1_ki == 1, name='Claim 1_ki_'+str(i))
        model.addConstr( obj_claim_1_ik == 1, name='Claim 1_ik_'+str(i))
'''        


In [120]:
# Variable Reduction
'''
# Claim 2
for sigma in serve_list:
    index = serve_list.index(sigma)
    if D_less_equal_than_W_for_i_in_sigma(sigma, drive_time, walk_time, n):
        for i in range(1,n+1):
            for k in range(1,n+1):
                if (i not in sigma) or (k not in sigma):
                    model.addConstr( y[i,index,k] == 0, name='Claim 2_'+str(i) +str(k)+str(index))
'''

In [121]:
 # Claim 3
'''
for sigma in serve_list:
    index = serve_list.index(sigma)
    for i in range(1,n+1):
        for k in range(1,n+1):
            c_first, c_last = find_shortest_path(i, sigma, k)
            if (walk_time[(i,c_first)] - drive_time[(i,c_first)]) >= wait_time[(c_first, index, k)]:
                model.addConstr( y[i,index,k] == 0, name='Claim 3_'+str(i) +str(k)+str(index))

'''


In [ ]:
# Claim 4
'''
for sigma in serve_list:
    index = serve_list.index(sigma)
    for i in range(1,n+1):
        for k in range(1,n+1):
            c_first, c_last = find_shortest_path(i, sigma, k)
            if (walk_time[(c_last,k)] - drive_time[(c_last,k)]) >= wait_time[(i, index, c_last)]:
                model.addConstr( y[i,index,k] == 0, name='Claim 4_'+str(i) +str(k)+str(index))
'''

In [100]:
'''
print("Claim 5 and Claim 6 should be incorported is ",D_less_equal_than_W(drive_time, walk_time, n))
'''

Claim 5 and Claim 6 should be incorported is  True


In [101]:
'''
# Added valid inequalities
# Only for D(i,k)<=W(i,k) 

# Claim 5
for i in range(1,n+1):
    for k in range(1, n+1):
        if i != k:
            obj_claim_5 = LinExpr(0)
                
            J_ik_idx = [ idx for idx in range(len(serve_list)) if (i in serve_list[idx]) and (k in serve_list[idx]) ]
            
            for s_id in J_ik_idx:
                for a in range(1,n+1):
                    for b in range(1, n+1):
                        obj_claim_5.add(y[a,s_id,b])
            model.addConstr( obj_claim_5 + x[i,k] <= 1, name='Claim 5_'+str(i)+'_'+str(k))

# Claim 6
for i in range(1,n+1):
    for k in range(1, n+1):
        if i < k:
            obj_claim_6 = LinExpr(0)
            for s_id in range(len(serve_list)):
                obj_claim_6.add(y[k,s_id,i] + y[i,s_id,k])
            model.addConstr( obj_claim_6 + x[i,k] + x[k,i] <= 1, name='Claim 6_'+str(i)+'_'+str(k))

'''

In [213]:
model.optimize()

print("\n\n optimal value:")
print(model.ObjVal)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 504 rows, 540841 columns and 3655901 nonzeros
Model fingerprint: 0xefac00fa
Variable types: 0 continuous, 540841 integer (540441 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-02, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 41 rows and 41 columns (presolve time = 5s) ...
Presolve removed 41 rows and 41 columns
Presolve time: 5.10s
Presolved: 463 rows, 540800 columns, 3655780 nonzeros
Variable types: 0 continuous, 540800 integer (540420 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.900000e+01   0.000000e+00     13s

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      0.0000000e+00     13s

Sifting complete


Root relaxation: infeasible, 350 it

AttributeError: Unable to retrieve attribute 'ObjVal'